## 프로그램 로직 설명 가이드

#### 특정 기업, 특정 년도의 사업보고서를 가져온다

In [14]:
import OpenDartReader

api_key = 'f2e08d4ed3de0ba3d5cbf59c04c223e02b1751a2'
dart = OpenDartReader(api_key)

str_expr = "report_nm.str.contains('사업보고서')"
stock_code = '005930'   # 삼성전자
start_date = '20230101'
end_date = '20231231'
# 정기공시(kind='A') 보고서 최종(final=True)
df_rpt = dart.list(stock_code, start_date, end_date, kind='A', final=True)
# 사업보고서
df_biz_rpt = df_rpt.query(str_expr)

In [15]:
df_rpt

,corp_code,corp_name,stock_code,corp_cls,report_nm,rcept_no,flr_nm,rcept_dt,rm
0,00126380,삼성전자,005930,Y,분기보고서 (2023.09),20231114002109,삼성전자,20231114,
1,00126380,삼성전자,005930,Y,반기보고서 (2023.06),20230814002534,삼성전자,20230814,
2,00126380,삼성전자,005930,Y,분기보고서 (2023.03),20230515002335,삼성전자,20230515,
3,00126380,삼성전자,005930,Y,사업보고서 (2022.12),20230307000542,삼성전자,20230307,연


In [16]:
df_biz_rpt

,corp_code,corp_name,stock_code,corp_cls,report_nm,rcept_no,flr_nm,rcept_dt,rm
3,00126380,삼성전자,005930,Y,사업보고서 (2022.12),20230307000542,삼성전자,20230307,연


#### 사업보고서에서 <재무제표 주석> 문서 가져오기

* 사업보고서에서 <재무제표 주석> 문서만 담겨있는 URL을 가져온다.

In [18]:
subreports = []
for i, row in df_biz_rpt.iterrows():
    rpt_name = row['report_nm']
    rpt_num = row['rcept_no']
    rpt_url = 'http://dart.fss.or.kr/dsaf001/main.do?rcpNo=' + rpt_num
    # 사업보고서의 Title 목록을 가져온다.
    df_doc = dart.sub_docs(rpt_num)     
    # "재무제표 주석" 문서를 가져온다 (연결재무제표 주석 문서 아님)
    df_notes = df_doc.loc[(df_doc.title.str.contains('주석')) & (~df_doc.title.str.contains('연결')),]
    rpt_title = df_notes.iloc[0]['title']
    rpt_notes_url = df_notes.iloc[0]['url']
    subreports.append({'corp_code' : row['corp_code'], 'stock_code' : row['stock_code'], 'corp_cls' : row['corp_cls'], 'corp_name' : row['corp_name'],
                        'rpt_num' : rpt_num, 'rpt_name' : rpt_name, 'rpt_url' : rpt_url, 
                        'doc_title' : rpt_title, 'doc_url' : rpt_notes_url, 'doc_result' : 'OK'})

In [19]:
rpt_url

'http://dart.fss.or.kr/dsaf001/main.do?rcpNo=20230307000542'

In [21]:
# 사업보고서 Title 목록
pd.options.display.max_colwidth = 150
df_doc

,title,url
0,사 업 보 고 서,http://dart.fss.or.kr/report/viewer.do?rcpNo=20230307000542&dcmNo=9040011&eleId=1&offset=800&length=4053&dtd=dart3.xsd
1,【 대표이사 등의 확인 】,http://dart.fss.or.kr/report/viewer.do?rcpNo=20230307000542&dcmNo=9040011&eleId=2&offset=28077&length=401&dtd=dart3.xsd
2,I. 회사의 개요,http://dart.fss.or.kr/report/viewer.do?rcpNo=20230307000542&dcmNo=9040011&eleId=3&offset=28482&length=110842&dtd=dart3.xsd
3,1. 회사의 개요,http://dart.fss.or.kr/report/viewer.do?rcpNo=20230307000542&dcmNo=9040011&eleId=4&offset=28581&length=24596&dtd=dart3.xsd
4,2. 회사의 연혁,http://dart.fss.or.kr/report/viewer.do?rcpNo=20230307000542&dcmNo=9040011&eleId=5&offset=53181&length=21796&dtd=dart3.xsd
5,3. 자본금 변동사항,http://dart.fss.or.kr/report/viewer.do?rcpNo=20230307000542&dcmNo=9040011&eleId=6&offset=74981&length=2712&dtd=dart3.xsd
6,4. 주식의 총수 등,http://dart.fss.or.kr/report/viewer.do?rcpNo=20230307000542&dcmNo=9040011&eleId=7&offset=77697&length=61001&dtd=dart3.xsd
7,5. 정관에 관한 사항,http://dart.fss.or.kr/report/viewer.do?rcpNo=20230307000542&dcmNo=9040011&eleId=8&offset=138702&length=608&dtd=dart3.xsd
8,II. 사업의 내용,http://dart.fss.or.kr/report/viewer.do?rcpNo=20230307000542&dcmNo=9040011&eleId=9&offset=139328&length=145237&dtd=dart3.xsd
9,1. 사업의 개요,http://dart.fss.or.kr/report/viewer.do?rcpNo=20230307000542&dcmNo=9040011&eleId=10&offset=139666&length=2406&dtd=dart3.xsd


In [22]:
# 재무제표 주석 문서와 URL
subreports

[{'corp_code': '00126380',
  'stock_code': '005930',
  'corp_cls': 'Y',
  'corp_name': '삼성전자',
  'rpt_num': '20230307000542',
  'rpt_name': '사업보고서 (2022.12)',
  'rpt_url': 'http://dart.fss.or.kr/dsaf001/main.do?rcpNo=20230307000542',
  'doc_title': '5. 재무제표 주석',
  'doc_url': 'http://dart.fss.or.kr/report/viewer.do?rcpNo=20230307000542&dcmNo=9040011&eleId=22&offset=970584&length=432459&dtd=dart3.xsd',
  'doc_result': 'OK'}]

* 재무제표 주석 문서의 Contents(HTML)를 가져온다

In [23]:
import pandas as pd
import sys
module_path = "D:\PythonProject\data-gatherer\dart_fs_notes"
sys.path.append(module_path)
import mydart

df_doc = pd.DataFrame(subreports)

for i, doc in df_doc.iterrows():
    val_a = val_b = 0.0
    section_title = None
    unit = None
    doc_result = doc['doc_result']
    doc_url = doc['doc_url']
    # 재무제표 주석 URL에 접속해서 해당 문서의 contents를 가져온다.                    
    soup = mydart.get_document(doc_url)

soup

<!DOCTYPE HTML PUBLIC "-//W3C//DTD HTML 4.01 Transitional//EN" "http://www.w3.org/TR/html4/loose.dtd">

<html style="border:0">
<head>
<title></title>
<meta content="ie=edge" http-equiv="X-UA-Compatible"/>
<meta content="gdi" http-equiv="X-UA-TextLayoutMetrics"/>
<meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
<link href="/css/report_xml.css" rel="stylesheet" type="text/css"/>
</head>
<body bgcolor="#FFFFFF">
<p><br/></p>
<p class="section-2"><a name="toc1">5. 재무제표 주석</a></p>
<p><br/></p>
<table class="nb" width="629">
<colgroup>
<col width="629"/>
</colgroup>
<tbody>
<tr>
<td align="CENTER" height="23" valign="TOP" width="629">제 54 기 : 2022년 12월 31일 현재</td>
</tr>
<tr>
<td align="CENTER" height="23" valign="TOP" width="629">제 53 기 : 2021년 12월 31일 현재</td>
</tr>
<tr>
<td align="CENTER" height="23" valign="TOP" width="629"><br/></td>
</tr>
<tr>
<td height="23" valign="TOP" width="629">삼성전자주식회사</td>
</tr>
</tbody>
</table>
<p><br/></p>
<p>1. 일반적 사항:<br/><br/>삼성전자주식회사(이

#### 재무제표 주석 문서의 구조를 파악하고 퇴직연금에 대해 설명한 Section을 찾아서 가져온다

* 재무제표 주석 문서 내의 소제목 번호를 가져온다

In [24]:
subtitle_list = mydart.get_subtitles(soup)
subtitle_list

['1. 일반적 사항:',
 '2. 중요한 회계처리방침:',
 '3. 중요한 회계추정 및 가정:',
 '4. 범주별 금융상품:',
 '5. 금융자산의 양도:',
 '6. 공정가치금융자산:',
 '7. 매출채권 및 미수금:',
 '8. 재고자산:',
 '9. 종속기업, 관계기업 및 공동기업 투자:',
 '10. 유형자산:',
 '11. 무형자산:',
 '12. 차입금:',
 '13. 사채:',
 '14. 순확정급여부채(자산):',
 '15. 충당부채:',
 '16. 우발부채와 약정사항:',
 '17. 계약부채:',
 '18. 자본금:',
 '19. 이익잉여금:',
 '20. 기타자본항목:',
 '21. 비용의 성격별 분류:',
 '22. 판매비와관리비:',
 '23. 기타수익 및 기타비용:',
 '24. 금융수익 및 금융비용:',
 '25. 법인세비용:',
 '26. 주당이익:',
 '27. 현금흐름표:',
 '28. 재무위험관리:',
 '29. 부문별 보고:',
 '30. 특수관계자와의 거래:',
 '31. 보고기간 후 사건']

* "퇴직연금" 운용에 관해 기술한 Section의 소제목을 찾는다

In [25]:
section_title = mydart.find_section_title(subtitle_list)
section_title

'14. 순확정급여부채(자산):'

* "퇴직연금" 운영에 관해 기술한 Section 부문만을 발췌한다.

In [26]:
section_page = mydart.get_section_html(soup, subtitle_list, section_title)
section_page

'14. 순확정급여부채(자산):\n   <br/>\n   <br/>\n   가. 보고기간종료일 현재 순확정급여부채(자산)의 내역은 다음과 같습니다.\n  </p>\n  <table class="nb" width="616">\n   <colgroup>\n    <col width="616"/>\n   </colgroup>\n   <tbody>\n    <tr>\n     <td align="RIGHT" height="20" width="616">\n      (단위 : 백만원)\n     </td>\n    </tr>\n   </tbody>\n  </table>\n  <table border="1" width="616">\n   <colgroup>\n    <col width="335"/>\n    <col width="141"/>\n    <col width="140"/>\n   </colgroup>\n   <thead>\n    <tr>\n     <th align="CENTER" height="20" width="335">\n      구     분\n     </th>\n     <th align="CENTER" height="20" width="141">\n      당기말\n     </th>\n     <th align="CENTER" height="20" width="140">\n      전기말\n     </th>\n    </tr>\n   </thead>\n   <tbody>\n    <tr>\n     <td height="20" valign="TOP" width="335">\n      기금이 적립된 확정급여채무의 현재가치\n     </td>\n     <td align="RIGHT" height="20" valign="TOP" width="141">\n      10,729,126\n     </td>\n     <td align="RIGHT" height="20" valign="TOP" width="140">\n      11,155

#### 탐색하고자 하는 항목을 검색해서 찾고 해당 금액을 가져온다.

* "퇴직연금" Section에서 사용된 금액 단위를 찾는다.

In [27]:
# 해당 Section에서 사용된 금액 단위
unit = mydart.find_unit_from_section(section_page)
unit

'백만원'

* "퇴직연금" Section에서 찾고자 하는 항목과 그 항목에 해당하는 당기말의 금액을 찾는다.
  (기본적으로 "표"로 기술된 형태에서 찾음)

In [28]:
# 당기말의 "확정급여채무"와 "사외적립자산" 금액 찾기
val_a, val_b, item_result = mydart.find_item_from_section(section_page)      
print(item_result)
print(val_a, val_b)

기금이 적립된 확정급여채무의 현재가치, 기금이 적립되지 않은 확정급여채무의 현재가치, 사외적립자산의 공정가치
10758672.0 15168895.0


#### 목표값 탐색한 결과

In [29]:
data_list = []
data_list.append({'corp_code' : doc['corp_code'], 'stock_code' : doc['stock_code'], 'corp_cls' : doc['corp_cls'], 'corp_name' : doc['corp_name'], 
                'rpt_num' : doc['rpt_num'], 'rpt_name' : doc['rpt_name'], 
                'doc_title' : doc['doc_title'], 'doc_result' : doc_result, 'section_title' : section_title, 
                'unit' : unit, 'val_a' : val_a, 'val_b' : val_b, 'item_result' : item_result, 'doc_url' : doc_url})
data_list

[{'corp_code': '00126380',
  'stock_code': '005930',
  'corp_cls': 'Y',
  'corp_name': '삼성전자',
  'rpt_num': '20230307000542',
  'rpt_name': '사업보고서 (2022.12)',
  'doc_title': '5. 재무제표 주석',
  'doc_result': 'OK',
  'section_title': '14. 순확정급여부채(자산):',
  'unit': '백만원',
  'val_a': 10758672.0,
  'val_b': 15168895.0,
  'item_result': '기금이 적립된 확정급여채무의 현재가치, 기금이 적립되지 않은 확정급여채무의 현재가치, 사외적립자산의 공정가치',
  'doc_url': 'http://dart.fss.or.kr/report/viewer.do?rcpNo=20230307000542&dcmNo=9040011&eleId=22&offset=970584&length=432459&dtd=dart3.xsd'}]